In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# @title Imports
!pip install tensorflow

import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from drive.MyDrive.stocks.pretrained_rl_ppo.scr.check_data import prepare_time_series
from drive.MyDrive.stocks.pretrained_rl_ppo.scr.dataset_builder import DatasetBuilderForYourColumns
from drive.MyDrive.stocks.pretrained_rl_ppo.scr.ppo_training import train, prepare_datasets, testing_simulation
from drive.MyDrive.stocks.pretrained_rl_ppo.scr.backtest_env import BacktestEnv, EnvConfig
from drive.MyDrive.stocks.pretrained_rl_ppo.scr.indicators import *

In [16]:
raw_df

,timestamp,Open,High,Low,Close,Volume


In [17]:
# @title Load and check data

# PATH_TO_DATA = "dummy_df.joblib"  # <-- provide path
PATH_TO_DATA = "/content/drive/MyDrive/stocks/Data/DOGEUSDT/raw_data/DOGEUSDT_1m_2024-01-01_to_2025-09-10.joblib"
START = "2024-01-01"
END = "2025-09-10"
TIMESTAMP_COL = "timestamp"          # column with timestamps

raw_df = joblib.load(PATH_TO_DATA)
df = prepare_time_series(raw_df, timestamp_col=TIMESTAMP_COL).loc[START:END]
print(f"From {df.index[0]} to {df.index[-1]}")

From 2024-01-01 00:00:00+00:00 to 2025-09-10 23:59:00+00:00


In [18]:
# @title Add indicators

# Базовые массивы (создаём один раз)
close  = df['Close'].to_numpy(np.float64)
high   = df['High'].to_numpy(np.float64)
low    = df['Low'].to_numpy(np.float64)
volume = df['Volume'].to_numpy(np.float64)

# Словарь для новых колонок
new_cols = {}
obv_arr = obv_numba(close, volume)

for i in range(35, 246, 7):
    ema_i = ema_numba(close, i)
    sma_i = sma_numba(close, i)
    adx_i = adx_numba(high, low, close, i)
    atr_i = atr_numba(high, low, close, i)
    rsi_i = rsi_numba(close, i)

    fast = i
    slow = i * 2
    sig  = i // 2  # вместо int(i/2)

    macd, macd_signal, macd_hist = macd_numba(close, fast, slow, sig)
    stoch_k, stoch_d = stoch_numba(high, low, close, i, i // 2)
    boll_mid, boll_up, boll_dn = bollinger_numba(close, i, 2.0)
    cci_i  = cci_numba(high, low, close, i)
    willr_i= williams_r_numba(high, low, close, i)
    mfi_i  = mfi_numba(high, low, close, volume, i)
    roc_i  = roc_numba(close, i)
    vwap_i = vwap_numba(high, low, close, volume, i)  # если это rolling VWAP на окне i

    new_cols.update({
        f"EMA_{i}":          ema_i,
        f"SMA_{i}":          sma_i,
        f"ADX_{i}":          adx_i,
        f"ATR_{i}":          atr_i,
        f"RSI_{i}":          rsi_i,
        f"MACD_{i}":         macd,
        f"MACD_signal_{i}":  macd_signal,
        f"MACD_hist_{i}":    macd_hist,
        f"STOCH_K_{i}":      stoch_k,
        f"STOCH_D_{i}":      stoch_d,
        f"BOLL_MID_{i}":     boll_mid,
        f"BOLL_UP_{i}":      boll_up,
        f"BOLL_DN_{i}":      boll_dn,
        f"OBV":              obv_arr,
        f"CCI_{i}":          cci_i,
        f"WILLR_{i}":        willr_i,
        f"MFI_{i}":          mfi_i,
        f"ROC_{i}":          roc_i,
        f"VWAP_{i}":         vwap_i,
    })

# Добавляем все столбцы разом — без фрагментации
df = pd.concat([df, pd.DataFrame(new_cols, index=df.index)], axis=1)

print(df.columns.to_list())

['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'EMA_35', 'SMA_35', 'ADX_35', 'ATR_35', 'RSI_35', 'MACD_35', 'MACD_signal_35', 'MACD_hist_35', 'STOCH_K_35', 'STOCH_D_35', 'BOLL_MID_35', 'BOLL_UP_35', 'BOLL_DN_35', 'OBV', 'CCI_35', 'WILLR_35', 'MFI_35', 'ROC_35', 'VWAP_35', 'EMA_42', 'SMA_42', 'ADX_42', 'ATR_42', 'RSI_42', 'MACD_42', 'MACD_signal_42', 'MACD_hist_42', 'STOCH_K_42', 'STOCH_D_42', 'BOLL_MID_42', 'BOLL_UP_42', 'BOLL_DN_42', 'CCI_42', 'WILLR_42', 'MFI_42', 'ROC_42', 'VWAP_42', 'EMA_49', 'SMA_49', 'ADX_49', 'ATR_49', 'RSI_49', 'MACD_49', 'MACD_signal_49', 'MACD_hist_49', 'STOCH_K_49', 'STOCH_D_49', 'BOLL_MID_49', 'BOLL_UP_49', 'BOLL_DN_49', 'CCI_49', 'WILLR_49', 'MFI_49', 'ROC_49', 'VWAP_49', 'EMA_56', 'SMA_56', 'ADX_56', 'ATR_56', 'RSI_56', 'MACD_56', 'MACD_signal_56', 'MACD_hist_56', 'STOCH_K_56', 'STOCH_D_56', 'BOLL_MID_56', 'BOLL_UP_56', 'BOLL_DN_56', 'CCI_56', 'WILLR_56', 'MFI_56', 'ROC_56', 'VWAP_56', 'EMA_63', 'SMA_63', 'ADX_63', 'ATR_63', 'RSI_63', 'MACD_63', 

In [19]:
# @title Разметка тренда

def get_intervals(df: pd.DataFrame, col: str, treshold: int = 1000) -> list[tuple[str, int]]:
    """
    Находит интервалы от сигнала 1 до сигнала -1.

    :param df: DataFrame с datetime-индексом и колонкой сигналов {-1, 0, 1}
    :param col: имя колонки с сигналами
    :return: список кортежей (начало_как_str 'YYYY-MM-DD', длина_в_строках)
    """
    intervals = []
    start_idx = None
    start_pos = None  # позиция для подсчета длины

    for pos, (idx, val) in enumerate(df[col].items()):
        if val == 1 and start_idx is None:
            start_idx = idx
            start_pos = pos
        elif val == -1 and start_idx is not None:
            length = pos - start_pos + 1
            if length > treshold:
                intervals.append((start_idx.strftime("%Y-%m-%d"), length))
            start_idx = None
            start_pos = None

    return intervals

df['EMA_2500'] = ema_numba(close, 2500)

buy_sig  = (df['Close'] > df['EMA_2500'])
sell_sig = (df['Close'] < df['EMA_2500'])

df['Signal_Rule'] = np.where(buy_sig, 1, np.where(sell_sig, -1, 0)).astype(np.int8)

INDEXES = get_intervals(df, 'Signal_Rule')
print(INDEXES)

[('2024-01-01', 1776), ('2024-01-10', 2541), ('2024-01-20', 2310), ('2024-01-26', 1222), ('2024-01-27', 1709), ('2024-01-29', 1642), ('2024-02-07', 1977), ('2024-02-08', 2010), ('2024-02-10', 1059), ('2024-02-12', 1321), ('2024-02-14', 2058), ('2024-02-18', 2343), ('2024-02-24', 2756), ('2024-02-26', 4784), ('2024-03-01', 2791), ('2024-03-03', 3124), ('2024-03-08', 1050), ('2024-03-09', 1259), ('2024-03-20', 2590), ('2024-03-23', 3404), ('2024-03-25', 1506), ('2024-03-27', 2636), ('2024-04-06', 4211), ('2024-04-20', 1491), ('2024-05-02', 1221), ('2024-05-03', 4396), ('2024-05-09', 1141), ('2024-05-13', 1183), ('2024-05-15', 1804), ('2024-05-17', 1377), ('2024-05-20', 2698), ('2024-05-25', 2070), ('2024-06-04', 2219), ('2024-06-25', 2011), ('2024-06-27', 1328), ('2024-06-30', 1144), ('2024-07-06', 1676), ('2024-07-12', 2389), ('2024-07-14', 2512), ('2024-07-16', 1866), ('2024-07-19', 3126), ('2024-07-21', 1846), ('2024-07-26', 2444), ('2024-08-08', 2154), ('2024-08-10', 1376), ('2024-08

In [ ]:
# @title Объяснение параметров и метрик
"""
PPO_UPDATES — количество крупных циклов обучения (policy updates). Каждый цикл включает:
- Сбор PPO_ROLLOUT шагов из PPO_N_ENV параллельных окружений.
- Разбиение собранных данных на мини-батчи PPO_BATCH_SIZE.
- PPO_EPOCHS проходов оптимизации по этим батчам.
Таким образом, общее число шагов взаимодействия со средой за весь запуск равно PPO_UPDATES × PPO_N_ENV × PPO_ROLLOUT.
На что влияет:
- Общее время и объём обучения. Большее значение → больше данных и gradient-обновлений → дольше тренировка, выше шанс достичь лучшей политики.
- Риск переобучения.При фиксированных данных каждая лишняя итерация может привести к “запоминанию” конкретных траекторий и ухудшению обобщающей способности.
- Использование вычислительных ресурсов. PPO_UPDATES напрямую задаёт количество итераций оптимизации, что определяет нагрузку на GPU/CPU.
Как подбирать:
- Базовая точка: 50 циклов — типичное стартовое значение для средних по сложности задач (≈1 млн шагов при 40 × 512 × 50).
- Если агент недоучивается (наградная кривая всё ещё растёт) — увеличьте параметр (например, до 100–200), чтобы дать модели больше времени на освоение среды.
- Если награда стабилизируется или падает до завершения обучения — уменьшайте PPO_UPDATES или используйте раннюю остановку по метрике валидации.
- Ориентир по шагам среды: выберите целевой объём взаимодействий (например, 2 млн шагов) и рассчитайте PPO_UPDATES = целевые_шаги / (PPO_N_ENV × PPO_ROLLOUT).
- Грамотная настройка PPO_UPDATES помогает балансировать между временем обучения, объёмом данных и качеством итоговой политики.


PPO_N_ENV - задаёт число окружений (environment instances), запускаемых одновременно. Каждый из них параллельно генерирует свои траектории взаимодействия с миром.
Зачем нужны несколько сред:
- Быстрая генерация данных. Одновременно работающие окружения собирают шаги опыта параллельно, что ускоряет обучение: за один и тот же реальный промежуток времени вы получаете PPO_N_ENV раз больше данных.
- Снижение корреляции. Последовательные шаги в одной среде сильно коррелированы. Запуск множества независимых эпизодов уменьшает зависимость между образцами, что делает оценку градиента более надёжной.
- Стабильное обучение. При большем разнообразии исходных состояний и траекторий политика быстрее покрывает пространство состояний, что помогает избежать локальных минимумов.
Как подбирать:
- Ограничения железа. Увеличение PPO_N_ENV потребляет больше CPU/GPU и памяти. Выбирайте значение, которое эффективно загружает доступные ресурсы (типично 8–64).
- Размер батча. Общее число шагов на одно обновление равно PPO_N_ENV × PPO_ROLLOUT. Если вам нужен крупный батч (например, для стабильности), проще увеличить PPO_N_ENV, чем сильно удлинять PPO_ROLLOUT.
- Время взаимодействия. Если среда медленная или взаимодействие дорогое, увеличивайте PPO_N_ENV, чтобы по максимуму распараллелить сбор данных.
- Когда уменьшать. Если компьютер не справляется (CPU загружен на 100%, задержки растут), снизьте число сред или используйте асинхронную схему (A3C, RLLib).
Правильное значение PPO_N_ENV — компромисс между скоростью обучения и доступными ресурсами; начинайте с числа, оптимально загружающего ваш сервер, и корректируйте в зависимости от стабильности и производительности обучения.


PPO_ROLLOUT - задаёт длину собранной траектории из каждого окружения перед одним обновлением политики.
Если PPO_N_ENV = 40 и PPO_ROLLOUT = 512, то за цикл обучения собирается 40 × 512 = 20 480 шагов взаимодействия.
На что влияет:
- Размер батча и скорость обновления. Длинный rollout увеличивает объём данных на обновление и реже вызывает оптимизацию, что делает каждое обновление более «массовым», но редким.
- Корреляция данных. Чем длиннее траектория, тем более последовательные (коррелированные) шаги попадают в выборку. Это может замедлить обучение,
особенно в задачах с высокой автокорреляцией.
- Обновление значений и преимущества. Rollout определяет глубину вычисления возвратов/GAE. Слишком короткий сегмент даёт «обрезанные» оценки,
а слишком длинный может приводить к устареванию данных, если политика за время сбора уже изменилась.
- Потребление памяти. Хранение длинных траекторий требует больше RAM/VRAM, что может ограничить максимальное значение на слабом оборудовании.
Как подбирать:
- Стартовое значение: 128–512 шагов — типичный диапазон для большинства задач в игре или управлении роботами.
- Баланс с PPO_N_ENV: поддерживайте общий размер батча (PPO_N_ENV × PPO_ROLLOUT) в разумных пределах (например, 8 000–30 000 шагов). Если увеличиваете число сред, rollout можно уменьшить.
- Если обучение шумное и нестабильное: попробуйте удлинить rollout, чтобы каждое обновление опиралось на большее количество данных.
- Если среда очень медленная: укорачивайте rollout и/или увеличивайте PPO_N_ENV, чтобы чаще обновлять политику и быстрее получать обратную связь.
- Анализ качества: отслеживайте кривые награды и скорость сходимости; оптимальное значение rollout — то, при котором прирост награды стабилен, а время до обновления не слишком велико.
Итог: PPO_ROLLOUT задаёт, сколько последовательных шагов собирает каждая среда до обновления. Он напрямую влияет на размер батча, корреляцию данных и частоту обновлений, поэтому выбирать его нужно в связке с количеством параллельных окружений и возможностями оборудования.


PPO_CLIP_RATIO — центральный гиперпараметр в алгоритме Proximal Policy Optimization. Его называют «шириной зажима» и обычно обозначают ε.
В PPO вычисляется отношение вероятностей действий новой политики и старой. Чтобы не позволять обновлённой политике слишком далеко «отклоняться» от старой, каждая величина
r(θ) проходит через функцию clip(r, 1-ε, 1+ε) и затем берётся минимум.
На что влияет:
- Стабильность обучения. Чем меньше PPO_CLIP_RATIO, тем более «осторожные» обновления: политика меняется медленно, но обучение устойчивее.
- Скорость и амплитуда обновлений.Больший параметр позволяет быстрее приспосабливаться к изменениям среды — но есть риск «разлета» градиентов и деградации.
- Баланс исследования/эксплуатации. При небольшом ε алгоритм дольше сохраняет свойства старой политики, что снижает риск забывания, но может замедлить улучшение.
Как подбирать:
- Стартовое значение: 0.1–0.3 (типичный базовый диапазон из статьи PPO).
- Если обучение нестабильно или награда сильно колеблется, уменьшите ε (например, 0.1 или 0.05).
- Если модель «залипает» и улучшается медленно, попробуйте увеличить (0.3 и выше), но контролируйте, чтобы потери и дивергенция не расходились.
- Иногда применяют адаптивную схему: отслеживают фактический KL-дивергенцию между старой и новой политиками и динамически корректируют PPO_CLIP_RATIO, но это усложняет реализацию.
Итого: PPO_CLIP_RATIO — это безопасный «коридор», внутри которого политика может двигаться за один шаг обновления. Правильная настройка этого параметра критична для того, чтобы обучение было и стабильным, и достаточно быстрым.


PPO_MAX_GRAD_NORM - задаёт верхнее ограничение на норму (длину) вектора градиента при обновлении параметров сети.
После вычисления градиентов они при необходимости «обрезаются» так, чтобы их норма не превышала это значение.
На что влияет:
- Стабильность обучения. Ограничение градиентов предотвращает резкие скачки параметров и защищает от взрывных градиентов, особенно при высокой скорости обучения.
- Сходимость. При корректном «клиппинге» градиента модель движется к минимуму более плавно, что способствует более надёжной сходимости.
- Скорость обучения. Слишком низкий порог может замедлить адаптацию, поскольку сильные градиенты будут «срезаться», а слишком высокий — не окажет должной стабилизации.
Как подбирать:
- Стартовое значение: 0.5–1.0 — типичный диапазон для большинства задач с PPO.
- Если наблюдаются взрывы градиента (loss резко скачет, параметры становятся nan), уменьшите значение.
- Если обучение идёт слишком медленно и градиенты редко достигают порога, можно слегка увеличить PPO_MAX_GRAD_NORM.
- Комбинируйте с learning rate: при повышении PPO_ACTOR_LR или PPO_CRITIC_LR может потребоваться более жёсткое ограничение градиента.
Итого: PPO_MAX_GRAD_NORM — «страховочный трос» для градиентов, удерживающий шаги оптимизации в разумных пределах.
Его грамотная настройка помогает избежать нестабильности и сохранить баланс между скоростью и надёжностью обучения.


PPO_TARGET_KL — целевое значение KL-дивергенции между старой и новой политиками. После каждого обновления алгоритм измеряет фактическую KL-дивергенцию и сравнивает её с целевым порогом.
На что влияет:
- Контроль шага обновления. Если реальная дивергенция превышает PPO_TARGET_KL, дальнейшие итерации оптимизации останавливаются или замедляются — это предотвращает слишком резкие изменения политики.
- Стабильность обучения. Поддержание дивергенции вблизи целевого значения способствует более плавной и предсказуемой траектории обучения.
- Компромисс между скоростью и надёжностью. Более низкий порог делает обучение осторожным, но может замедлить прогресс; более высокий — ускоряет адаптацию, но повышает риск нестабильности.
Как подбирать:
- Стартовое значение: 0.01 — распространённый базовый выбор.
- Если наблюдается «разлёт» политики (потери нестабильны, награда скачет), попробуйте уменьшить PPO_TARGET_KL, чтобы жёстче контролировать изменения.
- Если обучение идёт слишком медленно и KL практически не достигает порога, значение можно поднять (например, до 0.02–0.05), позволяя большей свободе обновлений.
- Адаптивные схемы: иногда вводят постепенное снижение порога по ходу обучения, чтобы сначала быстро исследовать, а потом стабилизироваться.
Итого: PPO_TARGET_KL служит «стоп-сигналом» для слишком больших отклонений новой политики от старой.
Правильно подобранный порог помогает сочетать быстрое обучение с контролируемой стабильностью, что особенно важно при длительных или чувствительных к разрывам задачах.


PPO_GAMMA (discount factor) — коэффициент дисконтирования будущих вознаграждений.
Значение ближе к 1 делает агент более «дальнозорким», учитывая долгосрочные последствия действий.
Более низкое значение заставляет агента концентрироваться на ближайшем вознаграждении.
Типичные значения: 0.99, 0.95 — выбираются в зависимости от глубины временного горизонта задачи.


PPO_LAM (lambda) — параметр обобщённой оценки преимущества (GAE), регулирующий баланс между bias и variance.
Значение 1.0 приводит к меньшему смещению, но более высокому разбросу.
Значение 0.0 повышает смещение, но уменьшает разброс, что делает обучение более «шумным».
Часто используется 0.95–0.97, но оптимальное значение зависит от особенностей среды и политики.
Практика подбора:
- Начать с общепринятых значений (gamma=0.99, lam=0.95).
- Выполнить поиск по сетке (grid search) или использованию систематических экспериментов, меняя по одному параметру.
- Анализировать метрики обучения (скорость сходимости, стабильность, итоговое качество политики) и при необходимости корректировать параметры.


МЕТРИКИ:

policy_loss = -0.01140
Отражает, насколько сильно обновление политики снижает целевую функцию. Небольшое отрицательное значение говорит о том,
что текущие обновления улучшают политику, но шаги достаточно осторожные. Если абсолютное значение постоянно растёт (становится более отрицательным),
модель активно адаптируется; если стремится к нулю — политика стабилизируется.

value_loss = 0.72330
Показывает ошибку критика при оценке ожидаемого вознаграждения. Чем меньше, тем точнее предсказания.
Значение выше ~0.7 может означать, что критик ещё не до конца обучен или задача сложная.
Если loss со временем снижается — критик становится точнее; рост может указывать на переобучение политики или слишком высокую скорость обучения критика.

entropy = 0.43223
Измеряет “случайность” действий (степень исследования). 0.43 — умеренная энтропия: политика сохраняет разнообразие, но уже не совершенно случайна.
По мере обучения энтропия обычно убывает: слишком быстрое падение — риск преждевременной детерминированности, слишком медленное — модель долго остаётся хаотичной.

approx_kl = 0.05351
Приблизительная KL-дивергенция между старой и новой политиками. Она показывает, насколько сильно изменяется политика после обновления.
Значение 0.0535 в пять раз превышает ваш целевой PPO_TARGET_KL = 0.01, что может сигнализировать о слишком резких шагах.
Если approx_kl стабильно выше порога, стоит уменьшить PPO_CLIP_RATIO или скорость обучения.

teacher_kl = 0.0457
Среднее значение KL-дивергенции между текущей политикой и teacher‑моделью. В коде собранные на каждой итерации t_kl складываются в список teacher_kls, а затем усредняются.
Значение ~0.046 говорит, что политика умеренно отклоняется от teacher: выше нуля (политика учится отличаться), но далеко от заданного лимита 0.1.

clip_fraction = 0.0124
Доля шагов, где отношение вероятностей действий вышло за пределы 1 ± PPO_CLIP_RATIO и была применена обрезка.
В коде эти события накапливаются в clipfracs, после чего берётся их среднее. Низкое значение (~1.2 %) означает,
что обновления оставались в пределах допустимого коридора, и политика изменялась достаточно плавно.

avg_return = -0.1172
Среднее возвращённое вознаграждение по собранной траектории за обновление.
В коде вычисляется как среднее значение traj.returns и записывается в метрики.
Отрицательная величина показывает, что агент в среднем пока приносит убыток — требуется дальнейшее обучение или настройка гиперпараметров.

Что означают изменения во времени
Падение policy_loss и value_loss — признак прогресса, но отслеживайте, чтобы value_loss не рос после изначального падения.
Снижение энтропии — нормальный процесс, но важно, чтобы она не опускалась слишком рано; в противном случае увеличьте PPO_C2.
Пики approx_kl выше целевого порога — сигнал, что обновления могут быть слишком агрессивными;
при регулярном превышении значения следует уменьшить PPO_ACTOR_LR, PPO_CLIP_RATIO или число эпох PPO_EPOCHS.

"""

In [ ]:
# @title Обучение на весах SL модели
SL_MODEL_CONFIG = "/content/drive/MyDrive/stocks/pretrained_rl_ppo/sl_weights/exp3/config.joblib"
PPO_SAVE_PATH = '/content/drive/MyDrive/stocks/pretrained_rl_ppo/ppo_weights/exp3'
CONFIG = joblib.load(SL_MODEL_CONFIG)

SPLITS = (0.80, 0.15, 0.05)
BEST_MODEL_PATH = "/content/drive/MyDrive/stocks/pretrained_rl_ppo/sl_weights/exp3/best_full_model_ep:48_f1:68.3.weights.h5"
BEST_BACKBONE_PATH = "/content/drive/MyDrive/stocks/pretrained_rl_ppo/sl_weights/exp3/best_backbone__ep:37_f1:68.1.weights.h5"
BEST_CRITIC_PATH = "" # нет потому что в SL мы не учим критика, вместо этого берем backbone
FEATURES = CONFIG['features']
ACCOUNT_F = CONFIG['state']

# @title PPO training parameters
PPO_SEQ_LEN = CONFIG['seq_len'] # ширина окна (такая же как при обучении SL)
PPO_FEATURES_DIM = len(FEATURES) + len(ACCOUNT_F)
PPO_NUM_ACTIONS = CONFIG['num_actions'] # кол-во возможных действий (такое же как при обучении SL)
PPO_UNITS_PER_LAYER = CONFIG['units_per_layer'] #  кол-во нейронов (такое же как при обучении SL)
PPO_DROPOUT = CONFIG['dropout']

# PPO params
PPO_UPDATES = 20         # число обновлений политики за цикл обучения (эпохи)
PPO_N_ENV = 30          # количество параллельных окружений для сбора данных
PPO_N_VAL = 5           # кол-во валидаций на один update
PPO_ROLLOUT = 512       # длина траектории (количество шагов), собираемой перед обновлением
PPO_ACTOR_LR = 3e-3     # скорость обучения актёра
PPO_CRITIC_LR = 1e-3    # скорость обучения критика
PPO_CLIP_RATIO = 0.5    # центральный гиперпараметр в алгоритме Proximal Policy Optimization. Его называют «шириной зажима»
PPO_C1 = 0.5            # коэффициент перед вкладом ошибки критика в функции потерь
PPO_C2 = 0.03           # коэффициент перед бонусом энтропии в функции потерь (чем больше, тем больше эксплорейшн)
PPO_GAMMA = 0.99        # (discount factor) — коэффициент дисконтирования будущих вознаграждений.
PPO_LAM = 0.95          # параметр обобщённой оценки преимущества (GAE), регулирующий баланс между bias и variance.
PPO_EPOCHS = 20         # количество проходов по батчу при обучении
PPO_BATCH_SIZE = 256    # размер мини-батча для SGD
PPO_TEACHER_KL = 0.1    # начальный вес KL‑регуляризации с teacher‑моделью
PPO_KL_DECAY = 0.95      # коэффициент экспоненциального затухания веса KL
PPO_TARGET_KL = 0.01    # целевое значение KL‑дивергенции для ранней остановки
PPO_MAX_GRAD_NORM = 0.5 # ограничение нормы градиента для стабилизации обучения
PPO_VAL_INTERVAL = 2    # периодичность валидации (через сколько циклов обучения)
PPO_DEBUG = True        # режим вывода отладочной информации

# INDEXES = [("2024-05-01", 1000), ("2024-07-01", 500)] # если хочешь учить на конкретных отрезках данных

# Подготавливаем данные и вычисляем статистики нормализации
ppo_train_df, ppo_val_df, ppo_test_df, _feat_stats = prepare_datasets(df,
                                                                      FEATURES,
                                                                      splits=SPLITS,
                                                                      norm_kind='minmax')


TRAIN_CFG = EnvConfig(
    mode=1,                             # лонг = 1, шорт = -1
    fee=0.0002,                         # комиссия
    spread=0.0   ,                      # задать, если цена- это midl_price (иначе 0)
    leverage=1.0,                       # плечо
    max_steps=PPO_ROLLOUT+PPO_SEQ_LEN,  # ограничение по длине эпизода
    reward_scale=80.0,                 # вес пнл награды (у пнл обычно маленькое значение 0,001)
    use_log_reward=False,               # применение вместо относительного пнл, логарифма пнл
    valid_time=60,                       # задержка после которой начинат начисляться штраф
    time_penalty=0.0001,                # штраф за бездействие (np.sqrt(max(0, wait_steps -5) * time_penalty))
    hold_penalty=0.0001,                # штраф за удержание (np.sqrt(max(0, hold_steps -5) * hold_penalty))
)

VAL_CFG = EnvConfig(
    mode=1,
    fee=0.0002,
    spread=0.0,
    leverage=1.0,
    max_steps=1000, #размер валидацилнной выборки
    reward_scale=80.0,
    use_log_reward=False,
    valid_time=60,
    time_penalty=0.0001,
    hold_penalty=0.0001,
)

actor, critic, train_log, val_log = train(
    ppo_train_df,
    ppo_val_df,
    TRAIN_CFG,
    VAL_CFG,
    PPO_FEATURES_DIM,
    FEATURES,
    seq_len = PPO_SEQ_LEN,
    teacher_weights = BEST_MODEL_PATH,
    critic_weights = BEST_CRITIC_PATH,
    backbone_weights = BEST_BACKBONE_PATH,
    save_path = PPO_SAVE_PATH,
    num_actions = PPO_NUM_ACTIONS,
    units_per_layer = PPO_UNITS_PER_LAYER,
    dropout = PPO_DROPOUT,
    updates = PPO_UPDATES,
    n_env = PPO_N_ENV,
    n_validations = PPO_N_VAL,
    rollout = PPO_ROLLOUT,
    actor_lr = PPO_ACTOR_LR,
    critic_lr = PPO_CRITIC_LR,
    clip_ratio = PPO_CLIP_RATIO,
    c1 = PPO_C1,
    c2 = PPO_C2,
    gamma = PPO_GAMMA,
    lam = PPO_LAM,
    epochs = PPO_EPOCHS,
    batch_size = PPO_BATCH_SIZE,
    teacher_kl = PPO_TEACHER_KL,
    kl_decay = PPO_KL_DECAY,
    max_grad_norm = PPO_MAX_GRAD_NORM,
    target_kl = PPO_TARGET_KL,
    val_interval = PPO_VAL_INTERVAL,
    index_ranges = INDEXES,
    debug = PPO_DEBUG
)

In [ ]:
# @title Fine tuning

# Параметры
SL_MODEL_CONFIG = "/content/drive/MyDrive/stocks/pretrained_rl_ppo/sl_weights/exp1/config.joblib"
PPO_SAVE_PATH = '/content/drive/MyDrive/stocks/pretrained_rl_ppo/ppo_weights/ft'
CONFIG = joblib.load(SL_MODEL_CONFIG)

SPLITS = (0.83, 0.12, 0.05)
BEST_PATH_ACTOR = "/content/drive/MyDrive/stocks/pretrained_rl_ppo/ppo_weights/exp2/actor_best_1.weights.h5"
BEST_CRITIC_PATH = "/content/drive/MyDrive/stocks/pretrained_rl_ppo/ppo_weights/exp2/critic_best_1.weights.h5"
BEST_BACKBONE_PATH = ""
FEATURES = CONFIG['features']
ACCOUNT_F = CONFIG['state']

# @title PPO training parameters
PPO_SEQ_LEN = CONFIG['seq_len'] # ширина окна (такая же как при обучении SL)
PPO_FEATURES_DIM = len(FEATURES) + len(ACCOUNT_F)
PPO_NUM_ACTIONS = CONFIG['num_actions'] # кол-во возможных действий (такое же как при обучении SL)
PPO_UNITS_PER_LAYER = CONFIG['units_per_layer'] #  кол-во нейронов (такое же как при обучении SL)
PPO_DROPOUT = CONFIG['dropout']

# INDEXES = [("2024-05-01", 1000), ("2024-07-01", 500)]

# PPO training parameters
PPO_UPDATES = 2
PPO_N_ENV = 10
PPO_ROLLOUT = 256
PPO_ACTOR_LR = 3e-3
PPO_CRITIC_LR = 1e-3
PPO_CLIP_RATIO = 1.5
PPO_C1 = 0.5
PPO_C2 = 0.02
PPO_EPOCHS = 10
PPO_BATCH_SIZE = 64
PPO_TEACHER_KL = 0.05
PPO_KL_DECAY = 0.5
PPO_TARGET_KL = 0.001
PPO_MAX_GRAD_NORM = 0.5
PPO_VAL_INTERVAL = 1
PPO_DEBUG = True
PPO_FUNE_TUNE = True

TRAIN_CFG = EnvConfig(
    mode=1,
    fee=0.0002,
    spread=0.0001,
    leverage=1.0,
    max_steps=PPO_ROLLOUT+PPO_SEQ_LEN,
    reward_scale=1.0,
    use_log_reward=False,
    time_penalty=0.0,
    hold_penalty=0.0,
)

VAL_CFG = EnvConfig(
    mode=1,
    fee=0.0002,
    spread=0.0001,
    leverage=1.0,
    max_steps=5000,
    reward_scale=1.0,
    use_log_reward=False,
    time_penalty=0.0,
    hold_penalty=0.0,
)

actor, critic, train_log, val_log = train(
    ppo_train_df,
    ppo_val_df,
    TRAIN_CFG,
    VAL_CFG,
    PPO_FEATURES_DIM,
    FEATURES,
    seq_len = PPO_SEQ_LEN,
    teacher_weights = BEST_PATH_ACTOR,
    critic_weights = BEST_CRITIC_PATH,
    backbone_weights = BEST_BACKBONE_PATH,
    save_path = PPO_SAVE_PATH,
    num_actions = PPO_NUM_ACTIONS,
    units_per_layer = PPO_UNITS_PER_LAYER,
    dropout = PPO_DROPOUT,
    updates = PPO_UPDATES,
    n_env = PPO_N_ENV,
    rollout = PPO_ROLLOUT,
    actor_lr = PPO_ACTOR_LR,
    critic_lr = PPO_CRITIC_LR,
    clip_ratio = PPO_CLIP_RATIO,
    c1 = PPO_C1,
    c2 = PPO_C2,
    epochs = PPO_EPOCHS,
    batch_size = PPO_BATCH_SIZE,
    teacher_kl = PPO_TEACHER_KL,
    kl_decay = PPO_KL_DECAY,
    max_grad_norm = PPO_MAX_GRAD_NORM,
    target_kl = PPO_TARGET_KL,
    val_interval = PPO_VAL_INTERVAL,
    # index_ranges = INDEXES,
    debug = PPO_DEBUG,
    fine_tune = PPO_FUNE_TUNE
)

In [ ]:
# @title Test simulation
ACTOR_WEIGHTS = "/content/drive/MyDrive/stocks/pretrained_rl_ppo/ppo_weights/exp2/actor_best_ep:12_rt:0.0743.weights.h5"
SL_MODEL_CONFIG = "/content/drive/MyDrive/stocks/pretrained_rl_ppo/sl_weights/exp3/config.joblib"
CONFIG = joblib.load(SL_MODEL_CONFIG)

START = "2024-01-20"
END = "2025-09-10"
TEST_FROM = "2025-05-01"

# Загружаем параметры из сохраненного конфига
PPO_SEQ_LEN = CONFIG['seq_len']
PPO_FEATURES_DIM = len(CONFIG['features']) + len(CONFIG['state'])
PPO_UNITS_PER_LAYER = CONFIG['units_per_layer']
PPO_DROPOUT = CONFIG['dropout']
PPO_NUM_ACTIONS = CONFIG['num_actions']
FEATURES = CONFIG['features']

dataframe = df.loc[START:END].copy()

_,_,ppo_test_df, _ = prepare_datasets(dataframe,
                                  FEATURES,
                                  test_from=TEST_FROM,
                                  norm_kind='minmax')

TEST_CFG = EnvConfig(
    mode=1,
    fee=0.0002,
    spread=0.0,
    leverage=1.0,
    max_steps=1e9,
    reward_scale=100.0,
    use_log_reward=False,
    valid_time=50,
    time_penalty=0.001,
    hold_penalty=0.001,
)

testing_simulation(
    ppo_test_df,
    ACTOR_WEIGHTS,
    PPO_SEQ_LEN,
    PPO_FEATURES_DIM,
    PPO_UNITS_PER_LAYER,
    PPO_DROPOUT,
    PPO_NUM_ACTIONS,
    FEATURES,
    TEST_CFG,
    debug=False
)